In [191]:
import pandas as pd
df1=pd.read_csv('train.csv')
df1.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [192]:
from sklearn.preprocessing import LabelEncoder
df2['Sex']=LabelEncoder().fit_transform(df1['Sex'])
df2['Fare']=LabelEncoder().fit_transform(df1['Fare'])
df3=df2.drop(['Fare','Parch','Ticket','Cabin','Name','Embarked','Pclass'],axis=1)
df3.fillna((df3.mean()),inplace=True)
target=df3.Survived
feature=df3.drop('Survived',axis=1)
feature

,Sex,Age
0,1,22.000000
1,0,38.000000
2,0,26.000000
3,0,35.000000
4,1,35.000000
...,...,...
886,1,27.000000
887,0,19.000000
888,0,29.699118
889,1,26.000000


In [193]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
scores=-1*cross_val_score(LogisticRegression(),feature,target,cv=6)
scores

array([-0.79865772, -0.77852349, -0.83221477, -0.77027027, -0.75675676,
       -0.78378378])

In [194]:
from sklearn.model_selection import train_test_split
xt,xv,yt,yv=train_test_split(feature,target,random_state=1)
from xgboost import XGBClassifier
XGBmodel=XGBClassifier(n_estimators=1000,learning_rate=0.05,n_jobs=3)
XGBmodel.fit(xt,yt,early_stopping_rounds=5,eval_set=[(xv,yv)],verbose=False)
output=XGBmodel.predict(xv)
XGBoutput=pd.DataFrame(data=output)
from sklearn.metrics import mean_squared_error
print(mean_squared_error(XGBoutput,yv))
XGBoutput

0.25112107623318386


,0
0,1
1,0
2,1
3,1
4,1
...,...
218,1
219,0
220,0
221,0


In [195]:
from sklearn.ensemble import RandomForestClassifier
RFCmodel=RandomForestClassifier()
RFCmodel.fit(xt,yt)
output=RFCmodel.predict(xv)
RFCoutput=pd.DataFrame(data=output)
print(mean_squared_error(RFCoutput,yv))


AttributeError: 'RandomForestClassifier' object has no attribute '_validate_data'

In [196]:
from sklearn.neighbors import KNeighborsClassifier
KNNmodel=KNeighborsClassifier()
KNNmodel.fit(xt,yt)
output=KNNmodel.predict(xv)
KNNoutput=pd.DataFrame(data=output)
print(mean_squared_error(KNNoutput,yv))
KNNoutput

0.2825112107623318


,0
0,0
1,0
2,1
3,1
4,1
...,...
218,1
219,0
220,0
221,0


In [197]:
from sklearn import svm
SVMmodel=svm.SVC()
SVMmodel.fit(xt,yt)
output=SVMmodel.predict(xv)
SVMoutput=pd.DataFrame(data=output)
print(mean_squared_error(SVMoutput,yv))
SVMoutput

0.4349775784753363


,0
0,0
1,0
2,0
3,0
4,1
...,...
218,0
219,0
220,0
221,0


In [198]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(
    objective='binary',
    boosting='gbdt',
    learning_rate = 0.05,
    max_depth = 8,
    num_leaves = 80,
    n_estimators = 400,
    bagging_fraction = 0.8,
    feature_fraction = 0.9)
    #reg_alpha = 0.2,
    #reg_lambda = 0.4)

lgb.fit(xt,yt)
output=lgb.predict(xv)
LGBMoutput=pd.DataFrame(data=output)
print(mean_squared_error(LGBMoutput,yv))
LGBMoutput

0.2600896860986547


,0
0,1
1,0
2,1
3,1
4,0
...,...
218,1
219,0
220,0
221,0


In [199]:
odf=pd.read_csv('test.csv')
from sklearn.preprocessing import LabelEncoder
odf['Sex']=LabelEncoder().fit_transform(odf['Sex'])
odf['Fare']=LabelEncoder().fit_transform(odf['Fare'])
odf=odf.drop(['Fare','Parch','Ticket','Cabin','Name','Embarked','SibSp','PassengerId','Pclass'],axis=1)
odf.fillna((odf.mean()),inplace=True)
odf

,Sex,Age
0,1,34.50000
1,0,47.00000
2,1,62.00000
3,1,27.00000
4,0,22.00000
...,...,...
413,1,30.27259
414,0,39.00000
415,1,38.50000
416,1,30.27259


In [200]:
fo=.predict(odf)
op=pd.DataFrame(data=fo)
op.to_csv('output.csv',index=False)
df=pd.read_csv('output.csv')
df

,0
0,0
1,0
2,0
3,0
4,1
...,...
413,0
414,1
415,0
416,0
